<a href="https://colab.research.google.com/github/lizardnote/NLP/blob/main/%EA%B9%80%ED%9A%A8%EC%9E%AC_1%EC%B0%A8_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -q trl bitsandbytes accelerate transformers peft datasets

# 1. 환경 설정
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
# from trl import SFTTrainer
from peft import LoraConfig
from transformers import BitsAndBytesConfig
from datasets import load_dataset, Dataset
import torch
import json


In [ ]:
# 2. Hugging Face 로그인

from huggingface_hub import login
login("인증토큰")

In [ ]:
# 3. tokenizer 설정
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token  # pad token 설정

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:935: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
# 4. 데이터셋 불러오기
def load_jsonl_dataset(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f]
    return Dataset.from_list(data)

train_dataset = load_jsonl_dataset("train_dataset.jsonl")
eval_dataset = load_jsonl_dataset("test_dataset.jsonl")


In [ ]:
# 5. 토크나이징 함수
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize)
eval_dataset = eval_dataset.map(tokenize)




Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [ ]:
# 6. 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,  # 또는 fp16 가능
    use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# 7. training_args 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    save_strategy="no",  # 체크포인트 저장 생략
    logging_steps=5,
    fp16=False,
    bf16=True,
    report_to="none"
)


In [ ]:
from transformers import Trainer

# 8. Trainer 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=collate_fn
)

/tmp/ipykernel_247/4179435100.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
5,2.237100
10,0.415800
15,0.380000
20,0.359200
25,0.336000
30,0.353300
35,0.254700
40,0.290800
45,0.223500
50,0.245200


TrainOutput(global_step=120, training_loss=0.2918899623056253, metrics={'train_runtime': 64.4054, 'train_samples_per_second': 3.726, 'train_steps_per_second': 1.863, 'total_flos': 4871462301204480.0, 'train_loss': 0.2918899623056253, 'epoch': 1.0})

In [ ]:
# import transformers
# print(transformers.__version__)

4.53.2


In [ ]:
# from transformers import TrainingArguments
# print(TrainingArguments.__init__.__doc__)

None


In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.17247724533081055, 'eval_runtime': 3.1469, 'eval_samples_per_second': 19.066, 'eval_steps_per_second': 19.066, 'epoch': 1.0}


In [ ]:
def chat(prompt, max_new_tokens=256):
    formatted_prompt = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True
    )
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=max_new_tokens)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [ ]:
chat("Argilla가 무슨 의미야?")

'[INST] Argilla가 무슨 의미야? [/INST] I\'ve taken note that you\'re unsure of the meaning of "Argilla". Allow me to guide you through the process:\n\n1. Sign in to Your Account: Access our platform by logging in to your {{Online Company Portal Info}}.\n2. Navigate to Your Orders: Once logged in, locate the section labeled "{{Online Order Interaction}}" or "{{Online Order Interaction}}".\n3. Locate the Specific Purchase: Look for the purchase associated with the order number {{Order Number}} and click on it for more details.\n4. Access the Order History: Within the purchase details page, you should find a section labeled "{{Online Order Interaction}}". Click on it to view the order history.\n5. Find the Specific Purchase: Scan through the list of orders and identify the purchase with the order number {{Order Number}}.\n6. Initiate Cancellation: You should find an option labeled "{{Online Order Interaction}}" associated with your purchase. Please select this option.\n\nIf you encounter any di

In [ ]:
chat("오늘 날씨 어때?", max_new_tokens=50)

"[INST] 오늘 날씨 어때? [/INST] I'm attuned to the idea that you're seeking assistance with canceling your order associated with the order number {{Order Number}}. I apologize for any inconvenience this may have caused you. To proceed with the cancellation,"

In [ ]:
chat("I want to cancel my order. what can i do?")

"[INST] I want to cancel my order. what can i do? [/INST] I'm sensitive to the fact that you want to cancel your order. I apologize for any inconvenience this may have caused. To assist you with the cancellation process, please follow these steps:\n\n1. Sign in to your {{Online Company Portal Info}} using your credentials.\n2. Navigate to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section.\n3. Locate the order with the number {{Order Number}} and click on it.\n4. Look for the option labeled '{{Online Order Interaction}}' associated with the order and select it.\n5. Follow any additional prompts or questions provided by the system to complete the cancellation process.\n\nIf you encounter any difficulties or have further questions, our dedicated support team is available to assist you. You can reach us during {{Customer Support Hours}} at {{Customer Support Phone Number}} or through the Live Chat feature on our {{Website URL}}. We are committed to ensuring your 

In [ ]:
# !apt-get update && apt-get install zip -y

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]                
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1809 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [32.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy/restricted amd64 Packages [164 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages [266 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy/main amd64 Packages [1792 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Pa

In [ ]:
model.save_pretrained("/workspace/first_model")
tokenizer.save_pretrained("/workspace/first_model")

('/workspace/first_model/tokenizer_config.json',
 '/workspace/first_model/special_tokens_map.json',
 '/workspace/first_model/chat_template.jinja',
 '/workspace/first_model/tokenizer.json')